# Purpose
研究婚外情的數據

(1) 該位外遇的可能性有多大？ 
(2) 影響外遇的原因是什麼？ 
(3) 可以根據分析結果說明如何降低外遇的發生嗎？

# 數據說明

* 婚外情數據即著名的“Fair’s Affairs”，取自於1969年《今日心理》（Psychology Today）所做的一個代表性調查
* 該數據從601個參與者身上收集了9個變量
* 變數包括一年來婚外情的頻率以及參與者性別、年齡、婚齡、是否有小孩、宗教信仰程度、學歷、職業，還有對婚姻的自我評分

#### affairs 受訪者在過去一年中進行外遇的頻率         
0 = 無         
1 = 一次       
2 = 兩次         
3 = 三倍         
7 = 4 - 10 次         
12 = 每月或更多         

#### gender          
0 = 女性         
1 = 男性         

#### age         
17.5 = 20 歲以下         
22.0 = 20 - 24         
27.0 = 25 - 29         
32.0 = 30 - 34         
37.0 = 35 - 39         
42.0 = 40 - 44         
47.0 = 45 - 49         
52.0 = 50 - 54         
57.0 = 55 或以上         

#### yearsmarried  婚姻時間                  
0.125 = 3 個月或更短         
0.417 = 4 - 6 個月         
0.750 = 6 個月 - 1 年         
1.500 = 1 - 2 年         
4.000 = 3 - 5 年         
7.000 = 6 - 8 年         
10.00 = 9 - 11 年         
15.00 = 12 年或更長時間         

#### children 孩子人數                  
0 = 無         
1 = 一個或多個         
         
#### religiousness 婚內的宗教信仰         
1 = 反宗教         
2 = 完全沒有         
3 = 輕微         
4 = 有點         
5 = 非常         

#### education 教育程度         
9.0 = 小學         
12.0 = 高中畢業         
14.0 = 一些大學         
16.0 = 大學畢業生         
17.0 = 一些畢業作品         
18.0 = 碩士學位         
20.0 = 博士、醫學博士或其他高級學位 

#### occupation 詳見data-descriptions.pdf

#### rating 對婚姻的自我評分 (5分制，1表示非常不幸福，5表示非常幸福）         

### <span style="color:#3498DB">Point: 請先將affairs變數轉為二值型因子'ynaffair'，外遇0次為0，外遇一次以上為1</span>

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [3]:
df = pd.read_csv('data/Affairs.csv')
df.head()

,affairs,gender,age,yearsmarried,children,religiousness,education,occupation,rating
0,0,male,37.0,10.00,no,3,18,7,4
1,0,female,27.0,4.00,no,4,14,6,4
2,0,female,32.0,15.00,yes,1,12,1,4
3,0,male,57.0,15.00,yes,5,18,6,5
4,0,male,22.0,0.75,no,2,17,6,3


(1) 該位外遇的可能性有多大？ 

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import statsmodels.api as sm

# 建立二元標籤：0 = 無外遇，1 = 有外遇（一次以上）
df["ynaffair"] = np.where(df["affairs"] > 0, 1, 0)

# One-hot encoding 處理類別變數
df_encoded = pd.get_dummies(df, columns=["gender", "children"], drop_first=True)

# 指定特徵與目標
features = ['age', 'yearsmarried', 'religiousness', 'education', 
            'occupation', 'rating', 'gender_male', 'children_yes']
X = df_encoded[features]
y = df_encoded["ynaffair"]

# 切分訓練集與測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 標準化特徵
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 建立 Logistic Regression 模型並訓練
model = LogisticRegression()
model.fit(X_train_scaled, y_train)

# 模型預測與評估
y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# 顯示結果
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_prob):.4f}")

# 預測個體外遇機率（自訂樣本）
# 範例輸入：35歲、婚齡10年、宗教3、教育16、職業5、滿意度4、男性、有無小孩
sample = np.array([[35, 10, 3, 16, 5, 4, 1, 0]])
sample_scaled = scaler.transform(sample)
sample_prob = model.predict_proba(sample_scaled)[0][1]

print(f"\n該位樣本預測外遇機率為：{sample_prob:.2%}")

Confusion Matrix:
[[88  3]
 [25  5]]

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.97      0.86        91
           1       0.62      0.17      0.26        30

    accuracy                           0.77       121
   macro avg       0.70      0.57      0.56       121
weighted avg       0.74      0.77      0.71       121

ROC AUC Score: 0.6938

該位樣本預測外遇機率為：21.03%


(2) 影響外遇的原因是什麼？

In [5]:
# 建立係數表
coef_df = pd.DataFrame({
    "Feature": features,
    "Coefficient": model.coef_[0],
    "Odds Ratio": np.exp(model.coef_[0])
}).sort_values(by="Odds Ratio", ascending=False)

# 顯示影響因子排序
print(coef_df)


         Feature  Coefficient  Odds Ratio
1   yearsmarried     0.355532    1.426939
7   children_yes     0.236760    1.267137
6    gender_male     0.211414    1.235423
4     occupation     0.087684    1.091643
3      education     0.017908    1.018069
0            age    -0.305045    0.737090
2  religiousness    -0.427931    0.651857
5         rating    -0.480698    0.618352


| Feature       | Coefficient | Odds Ratio | 解釋               |
| ------------- | ----------- | ---------- | ---------------- |
| yearsmarried  | +           | >1         | 結婚年數越久，越可能外遇     |
| children\_yes | +           | >1         | 有小孩略可能提高外遇風險     |
| rating        | –           | <1         | 婚姻滿意度高，外遇機率下降    |
| religiousness | –           | <1         | 越虔誠，外遇機率明顯下降     |
| gender\_male  | +           | >1         | 男性傾向外遇的機率略高（如為正） |
| age           | –           | <1         | 年齡越高，外遇傾向下降      |


(3) 可以根據分析結果說明如何降低外遇的發生嗎？

In [6]:
# 建立一組基礎特徵值（中性樣本）
# 特徵順序：[age, yearsmarried, religiousness, education, occupation, rating, gender_male, children_yes]
base = np.array([[35, 10, 2, 16, 5, 3, 1, 0]])  # 中度虔誠、滿意度普通

# 定義模擬條件
scenarios = {
    "原始狀態": base,
    "增加婚姻滿意度（3→5）": base.copy(),
    "提高宗教虔誠度（2→4）": base.copy(),
    "雙重提升：滿意度+虔誠度": base.copy()
}

# 修改對應欄位：rating=5、religiousness=4
scenarios["增加婚姻滿意度（3→5）"][0, 5] = 5
scenarios["提高宗教虔誠度（2→4）"][0, 2] = 4
scenarios["雙重提升：滿意度+虔誠度"][0, 5] = 5
scenarios["雙重提升：滿意度+虔誠度"][0, 2] = 4

# 計算每個情境下的外遇機率
print("各情境下預測外遇機率：\n")
for label, data in scenarios.items():
    scaled = scaler.transform(data)
    prob = model.predict_proba(scaled)[0][1]
    print(f"{label:<20} → 預測外遇機率：{prob:.2%}")


各情境下預測外遇機率：

原始狀態                 → 預測外遇機率：37.30%
增加婚姻滿意度（3→5）         → 預測外遇機率：19.84%
提高宗教虔誠度（2→4）         → 預測外遇機率：22.26%
雙重提升：滿意度+虔誠度         → 預測外遇機率：10.65%


| 如何降低外遇的發生                | 建議內容                                                   |
|----------------------------|-------------------------------------------------------------|
| 提升婚姻滿意度（最顯著因子） | 鼓勵溝通、情感支持、共同成長。                                 |
| 建立或強化宗教/道德支持     | 社群歸屬與價值導向有助降低外遇風險。                           |
| 關注婚齡變長的風險         | 定期檢視婚姻狀況、製造新鮮感與連結。                           |
